<img src="img/dh_logo.png" align="right" width="50%">

# Desafio 3

# Santander Kaggle
## Customer Transaction Prediction

<img src="img/santander.png"  >

### Neste desafio será necessário prever a variável target do DataSet

Para acessar e participar desta competição no Kaggle [clique aqui](https://www.kaggle.com/c/santander-customer-transaction-prediction/overview)

Para baixar os dados [clique aqui](https://www.kaggle.com/c/santander-customer-transaction-prediction/data)

## Santander: Overview da proposta do desafio

No Santander, nossa missão é ajudar pessoas e empresas a prosperar. Estamos sempre procurando maneiras de ajudar nossos clientes a entender sua saúde financeira e identificar quais produtos e serviços podem ajudá-los a atingir suas metas monetárias.

Nossa equipe de ciência de dados está desafiando continuamente nossos algoritmos de aprendizado de máquina, trabalhando com a comunidade global de dados científicos para garantir que possamos identificar com mais precisão novas maneiras de resolver nosso desafio mais comum, problemas de classificação binária como: um cliente está satisfeito? Um cliente comprará este produto? Um cliente pode pagar este empréstimo?

Neste desafio, convidamos a Kagglers a nos ajudar a identificar **quais clientes farão uma transação específica no futuro, independentemente do volume de dinheiro transacionado**. Os dados fornecidos para esta competição têm a mesma estrutura que os dados reais que temos disponíveis para resolver este problema.

# Algumas possibilidades a serem consideradas para desenvolvimento neste Desafio 3:

1. Explorar as variáveis explicatórias do problema e entender como elas influenciam na variável target


2. Identificar possíveis soluções para resolver o desbalancemanto de classes da variável target


3. Entender a explicabilidade dos dados utilizando diferentes técnicas não supervisionadas de Clustering e medir como os modelos respondem 


4. Buscar algumas formas de selecionar as melhores variáveis para classificar as fraudes e medir os resultados em um modelo preditor


5. Verificar se a redução de dimensionalidade pode ajudar na explicação das fraudes, com suficiente nível de informação, e medir os resultados em um modelo preditor


6. Submeter o melhor resultado na plataforma Keggle. Para isso, verifique qual a métrica adotada pelo examinador do desafio



You are provided with an anonymized dataset containing numeric feature variables, the binary target column, and a string ID_code column.

The task is to predict the value of target column in the test set.

### Santander DataFrame

In [ ]:
# import packages
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
# open train df
df = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/train.csv')

In [ ]:
df_test = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/test.csv')

In [ ]:
# check if there are missing values
# verificando se existem dados nulos
import missingno as msno
msno.matrix(df)

In [ ]:
# print df head
df.head()

In [ ]:
# print df shape
df.shape

200000 rows and 202 columns.

In [ ]:
# print df info
df.info()

In [ ]:
#describe the df variables
df.describe().T

In [ ]:
df.target.value_counts()

In [ ]:
sns.countplot(y = df.target)

In [ ]:
## correlation between target variable and the other columns
df_corr = df.corr().round(2)['target']

In [ ]:
df_corr.sort_values(ascending =True)

The variables aren't very correlated with the target value. As there are many variables I'll conduct a PCA to reduce the dataset dimensionality.

## PCA

In [ ]:
df.head()

In [ ]:
# I'll drop the ID_code and target variables of the dataset to do the PCA with the other variables.
#df_pca = df.drop(['ID_code'], axis = 1)
X_train = df.drop(['ID_code', 'target'], axis = 1)
X_test = df.drop(['ID_code', 'target'], axis = 1)

In [ ]:
# importando as bibliotecas
from sklearn.preprocessing import StandardScaler 

# instanciando a variável
sc = StandardScaler() 


# ajustando com os dados de treino
X_train = sc.fit_transform(X_train) 
X_test = sc.fit_transform(X_test)

In [ ]:
# importando as bibliotecas
from sklearn.decomposition import PCA

# instanciando o modelo
pca = PCA(n_components = 2)

# ajustando com os modelos de treino
X_train_pca = pca.fit_transform(X_train)

# transformando os dados de teste
X_test_pca = pca.transform(X_test)

In [ ]:
X_train_pca

In [ ]:
# Plotando um gráfico de dispersão entre as duas variáveis criadas pelo PC
# para identificaçào de como o PCA distribuiu as categorias
df2 = df.copy(deep=True)
# criação de duas novas colunas com as 2 dimensões do PCA
df2['PCA1'] = X_train_pca[:, 0]
df2['PCA2'] = X_train_pca[:, 1]
# criação de duas novas colunas com as 2 dimensões do PCA
df2['PCA1_test'] = X_test_pca[:, 0]
df2['PCA2_test'] = X_test_pca[:, 1]

In [ ]:
fig, axs = plt.subplots(1,2)
# plotando uma dispersão das novas colunas diferenciando as espécies
sns.lmplot('PCA1', 'PCA2', hue='target', data=df2, fit_reg=False);
# plotando uma dispersão das novas colunas diferenciando as espécies
sns.lmplot('PCA1_test', 'PCA2_test', hue='target', data=df2, fit_reg=False);

### Regressão Logística

In [ ]:
df.target

In [ ]:
# importando as bibliotecas
from sklearn.linear_model import LogisticRegression

# instanciando o modelo 
classifier = LogisticRegression(random_state = 42)
# ajustando o modelo
classifier.fit(X_train_pca, df.target) 

In [ ]:
# predição de valores com dados de teste com a Regressão Logística
y_pred_pca = classifier.predict(X_train_pca) 

In [ ]:
# plotando a Matriz de Confusão entre os valores reais e preditos

# importando a biblioteca
from sklearn.metrics import confusion_matrix 
# plotando a matriz
cm = confusion_matrix(df.target, y_pred_pca) 
cm

In [ ]:
sns.heatmap(cm,annot=True,cbar=False, xticklabels='auto' )

https://medium.com/@alepukhova526/principal-component-analysis-for-logistic-regression-with-scikit-learn-869b04b2f923

In [ ]:

# importar a funcao
from sklearn import metrics

# plotando a curva ROC
y_pred_proba_pca = classifier.predict_proba(X_train_pca)[::,1]
fpr, tpr, _ = metrics.roc_curve(df.target,  y_pred_proba_pca)
auc = metrics.roc_auc_score(df.target, y_pred_proba_pca)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
plt.legend(loc=4)

The PCA didn't perform very well on data compression.

## Classification

In [ ]:
# importando as bibliotecas
from sklearn.preprocessing import StandardScaler 

# instanciando a variável
sc = StandardScaler() 


# ajustando com os dados de treino
X= df.drop(['ID_code', 'target'], axis = 1)
X = sc.fit_transform(X) 
y= df.target

In [ ]:
# import the function
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [ ]:
print(len(y_train))
print(len(X_train))
print(len(y_test))
print(len(X_test))

In [ ]:
# importar a funcao
from sklearn.linear_model import LogisticRegression

# isntanciar o modelo
clf = LogisticRegression(dual = False, max_iter = 5000)

# ajustar aos dados de treino
clf.fit(X_train, y_train)

# predições para os dados de teste
y_pred = clf.predict(X_test)

In [ ]:
# importar a funcao
from sklearn import metrics

confusion_matrix(y_test, y_pred)

In [ ]:
# chamando a função da matriz de confusão
metrics.confusion_matrix

In [ ]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.9132

Precision: 0.6783854166666666

Recall: 0.25920398009950246

In [ ]:
# plotando a curva ROC
y_pred_proba = clf.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
plt.legend(loc=4)

auc = 0.859

In [ ]:
len(df)

In [ ]:
# fazendo um undersampling da classe com output zero (em maior número)
df_sample=df.sample(n=10000)

In [ ]:
# importando as bibliotecas
from sklearn.preprocessing import StandardScaler 

# instanciando a variável
sc = StandardScaler() 


# ajustando com os dados de treino
X_sp= df_sample.drop(['ID_code', 'target'], axis = 1)
X_sp = sc.fit_transform(X_sp) 
y_sp= df_sample.target

from sklearn.model_selection import train_test_split

Xsp_train, Xsp_test, ysp_train, ysp_test = train_test_split(X_sp, y_sp, test_size = 0.2, random_state = 42, stratify = y_sp)

In [ ]:
print(len(Xsp_train))
print(len(Xsp_test))

In [ ]:
# importando as bibliotecas dos modelos classificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

# definindo uma lista com todos os modelos
classifiers = [
    KNeighborsClassifier(),
    GaussianNB(),
    LogisticRegression(dual=False,max_iter=5000),
    SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier()]

# rotina para instanciar, predizer e medir os resultados de todos os modelos
for clf in classifiers:
    # instanciando o modelo
    clf.fit(Xsp_train, ysp_train)
    # armazenando o nome do modelo na variável name
    name = clf.__class__.__name__
    # imprimindo o nome do modelo
    print("="*30)
    print(name)
    # imprimindo os resultados do modelo
    print('****Results****')
    ysp_pred = clf.predict(Xsp_test)
    print("Accuracy:", metrics.accuracy_score(ysp_test, ysp_pred))
    print("Precision:", metrics.precision_score(ysp_test, ysp_pred))
    print("Recall:", metrics.recall_score(ysp_test, ysp_pred))
    
     # plotando a curva ROC
    y_pred_proba = clf.predict_proba(X_test)[::,1]
    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
    auc = metrics.roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label=name+", auc="+str(auc))
    plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
    plt.legend(loc=4)

### Balancing

In [ ]:
# definindo variáveis para cada uma das classes
df_0 = df[df.target == 0]
df_1 = df[df.target==1]

In [ ]:
print(len(df_0))
print(len(df_1))

In [ ]:
# undersampling
df_0=df_0.sample(len(df_1))

In [ ]:
df_concat = pd.concat([df_0,df_1])
df_concat.target.value_counts()

In [ ]:
# ajustando com os dados de treino
X= df_concat.drop(['ID_code', 'target'], axis = 1)
X = sc.fit_transform(X) 
y = df_concat.target

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [ ]:
# ignorando os warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# importnado as bibliotecas com os modelos classificadores

# definindo uma lista com todos os classificadores
classifiers = [
    KNeighborsClassifier(),
    GaussianNB(),
    LogisticRegression(),
    #SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier()]

# definindo o tamanho da figura para o gráfico
plt.figure(figsize=(12,8))

# rotina para instanciar, predizer e medir os rasultados de todos os modelos
for clf in classifiers:
    # instanciando o modelo
    clf.fit(X_train, y_train)
    # armazenando o nome do modelo na variável name
    name = clf.__class__.__name__
    # imprimindo o nome do modelo
    print("="*30)
    print(name)
    # imprimindo os resultados do modelo
    print('****Results****')
    y_pred = clf.predict(X_test)
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    print("Precision:", metrics.precision_score(y_test, y_pred))
    print("Recall:", metrics.recall_score(y_test, y_pred))
    
    
    # plotando a curva ROC
    y_pred_proba = clf.predict_proba(X_test)[::,1]
    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
    auc = metrics.roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label=name+", auc="+str(auc))
    plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
    plt.legend(loc=4)

## Gaussian NB

In [ ]:
# importar a funcao
from sklearn.naive_bayes import GaussianNB

# isntanciar o modelo
classifier = GaussianNB()

# ajustar aos dados de treino
classifier.fit(X_train, y_train)

# predições para os dados de teste
y_pred = clf.predict(X_test)

In [ ]:
metrics.confusion_matrix

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
# plotando a curva ROC
y_pred_proba = classifier.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
plt.legend(loc=4)

In [ ]:
# Verificando assimetria:
from scipy import stats

# choose numeric features
numeric_feats = df_concat.dtypes[df_concat.dtypes !="object"].index

skewed_feats = df_concat[numeric_feats].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending = False)

print("\nAssimetria: \n")
skew_df = pd.DataFrame({'Skew' :skewed_feats})
skew_df.head(20)

In [ ]:
norm = np.linalg.norm(df.var_179)
normal_array = df.var_179/norm
normal_array.skew()